## Client Collaboration
First, let's set up our client. Fill in the same username you used in the previous notebook to generate your certificate.

In [1]:
import securexgboost as xgb
from Utils import *
server_ip = "104.211.33.210:50052" # TODO: fill in the IP of the enclave server

In [2]:
# Get variables from previous notebooks
%store -r 
# PUB_KEY
# CERT_FILE
# KEY_FILE
# enc_training_data
# # enc_test_data

Next, centralize the data by send your training and test data to the person controlling the enclave server.

In [ ]:
# TODO: Utils function to scp
transfer_data(enc_training_data, server_ip)
transfer_data(enc_test_data, server_ip)

Once everyone has sent their data to the server, we can initialize our client.

In [3]:
# Run this cell to initialize your client
# TODO: fill out `client_list`
xgb.init_client(user_name=username, client_list=["chief", "chester"], sym_key_file=KEY_FILE, priv_key_file=PUB_KEY, cert_file=CERT_FILE, remote_addr=server_ip)

In [11]:
enc_training_data

'/home/mc2/risecamp/mc2/tutorial/data/chief_train.enc'

In [4]:
# Verify that the enclave has been set up correctly
xgb.attest()

Once we've authenticated the enclave, we can begin making requests to the enclave server. MC<sup>2</sup> enables users to make requests through a Python API, but will only execute requests once all users in the collaboration have submitted the same request. Consequently, users must submit the exact same requests in the exact order if they want to collaboratively compute. 

Let's first prepare for training by loading everyone's encrypted training data within the enclave.

In [5]:
# TODO: fill in usernames and training data paths
dtrain = xgb.DMatrix({username: enc_training_data})

In [6]:
# Set parameters
params = {
        "tree_method": "hist",
        "n_gpus": "0",
        "objective": "binary:logistic",
        "min_child_weight": "1",
        "gamma": "0.1",
        "max_depth": "3",
        "verbosity": "1" 
}
num_rounds = 5
booster = xgb.train(params, dtrain, num_rounds)

Training...


In [7]:
# Load everyone's test data
dtest1 = xgb.DMatrix({username: enc_test_data})
# dtest2 = xgb.DMatrix({username: enc_test_data})
# dtest3 = xgb.DMatrix({username: enc_test_data})

In [8]:
# Allow everyone to get their predictions
enc_preds, num_preds = booster.predict(dtest1, decrypt=False)
# enc_preds, num_preds = booster.predict(dtest, decrypt=False)
# enc_preds, num_preds = booster.predict(dtest, decrypt=False)

In [9]:
# Decrypt our predictions
preds = booster.decrypt_predictions(enc_preds, num_preds)
print(preds)

[0.10320058 0.8738707  0.10320058 ... 0.72566205 0.10053914 0.72566205]
